# Part I. ETL Pipeline for Pre-Processing the Files

## The following code does PRE-PROCESS the files by combines all the event_data into a single csv file

#### Import Python packages 

In [147]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [148]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [149]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [150]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster (Cassandra database connection)

In [151]:
# Make a connection to a Cassandra instance your local machine 

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [152]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
    )
except Exception as e:
    print(e)

#### Set Keyspace

In [153]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)
    


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




# Create queries 

## Query 1
## Give me the artist, song title and song's length in the music app history that was heard during sessionId=338, and itemInSession=4

### Create table (table name: music_sesssion_library)

In [154]:
# Drop table first
session.execute("DROP TABLE IF EXISTS music_session_library")

#query for creating a table
# PRIMARY KEY:
# Partition: sessionId. 
# Clustering columns: itemInSession
# SessionId and itemInSession should uniquely identify each music event.
# Song and artist should NOT be included in the Primary Key because the user can listen to the same song multiple times in a session.
query = "CREATE TABLE IF NOT EXISTS music_session_library "
query = query + "(sessionId int, itemInSession int, artist text, song text, length float, \
                  PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)                     

### Insert into table

In [155]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Insert statement into the `query` variable
        query = "INSERT INTO music_session_library (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## Insert statement for the table (sessionId is 9th column in CSV, so line[8], and so on)
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### SELECT (Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4)

In [156]:
query="select artist, song, length from music_session_library WHERE sessionId=338 and itemInSession=4"

try:
    rows=session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


## Query 2

## Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

### Create table (table name: user_sesssion_library)

In [157]:
# Drop table first
session.execute("DROP TABLE IF EXISTS user_session_library")

#query for creating a table
# PRIMARY KEY:
# Partition: sessionId, userId
# Clustering columns: itemInSession
# sessionId and itemInSession should uniquely identify each music event
# itemInSession as Clustering column to get desired sorting
query = "CREATE TABLE IF NOT EXISTS user_session_library "
query = query + "(sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, \
                  userId int, PRIMARY KEY ((sessionId, userId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e) 

### Insert into table

In [158]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Insert statement into the `query` variable
        query = "INSERT INTO user_session_library (sessionId, itemInSession, artist, song, firstName, lastName, userId)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Insert statement for the table (artist is 1th column in CSV, so line[0], and so on)
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], line[1], line[4], int(line[10])))

#### SELECT (Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182)

In [159]:
query="select artist, song, firstName, lastName from user_session_library WHERE userId=10 and sessionId=182"

try:
    rows=session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


## Query 3

## Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

### Create table (table name: song_library)

In [160]:
# Drop table first
session.execute("DROP TABLE IF EXISTS song_library")

#query for creating a table
# PRIMARY KEY:
# Partition: song 
# Clustering columns: userId
# This Primary Key setup will get each unique song-user combo which will answer the question.
query = "CREATE TABLE IF NOT EXISTS song_library "
query = query + "(firstName text, lastName text, userId int, song text,\
PRIMARY KEY (song, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e) 

### Insert into table

In [161]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Insert statement into the `query` variable

        query = "INSERT INTO song_library (firstName, lastName, userId, song)"
        query = query + "VALUES (%s, %s, %s, %s)"
        ## Insert statement for the table (firstName is 2nd column in CSV, so line[1], and so on)
        session.execute(query, (line[1], line[4], int(line[10]), line[9]))

#### SELECT (Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own)

In [162]:
query="select firstName, lastName from song_library WHERE song='All Hands Against His Own' "

try:
    rows=session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [163]:
query = "drop table music_session_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table user_session_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table song_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [164]:
session.shutdown()
cluster.shutdown()